In [ ]:
import warnings
from typing import List

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from autumn.core.project import get_project
from autumn.settings import Region, Models
from autumn.core import inputs
from autumn.core.utils.display import pretty_print
from autumn.core.inputs.tb_camau import queries
from autumn.core.inputs import get_death_rates_by_agegroup
import pathlib

In [ ]:
warnings.filterwarnings("ignore")
pd.options.plotting.backend = "plotly"

In [ ]:
region = Region.CAMAU
model = Models.TBD2
p = get_project(model, region, reload=True)
baseline_params = p.param_set.baseline
plots = p.plots
iso3 = baseline_params['country']['iso3']
source_iso3 = baseline_params['age_mixing']['source_iso3']
age_adjust = baseline_params['age_mixing']['age_adjust']
age_string_map = {
    0: "0-4",
    5: "5-14",
    15: "15-34",
    35: "35-49",
    50: "50+",
}

## Population

In [ ]:
modelled_age_groups = baseline_params["age_breakpoints"]
print(f"Modelled age groups are {modelled_age_groups}")

In [ ]:
csv_path = pathlib.Path('camau.csv')

In [ ]:
pop_df = pd.read_csv(csv_path)
pop_df = pop_df.set_index(['year'])
pop_df['population'].plot()

### Birth rate

In [ ]:
birth_rates, years = inputs.get_crude_birth_rate("VNM")
birth_rates = [b / 1000.0 for b in birth_rates]  # Birth rates are provided / 1000 population
birth_rates_series = pd.Series(birth_rates, index=years)
br_fig= px.line(birth_rates_series)
br_fig.update_traces(mode='markers+lines')
br_fig.update_layout(
    title="Crude birth rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude birth rate",
    showlegend=False,
)

### Life expectancy

In [ ]:
# Currently used in model:
death_rates_from_db = get_death_rates_by_agegroup(modelled_age_groups, "VNM")
death_rates_df = pd.DataFrame(death_rates_from_db[0], index=death_rates_from_db[1])
life_exp_at_age_df = 1.0 / death_rates_df
death_rates_df.plot()
life_exp_at_age_df.plot(log_y=True)

In [ ]:
# Query and visualize the crude birth rate from database
years, death_rates = queries.get_camau_death_rate()
death_rates = [b / 1000.0 for b in death_rates]  # Birth rates are provided / 1000 population
death_rates_series = pd.Series(death_rates, index=years)
dr_fig= px.line(death_rates_series)
dr_fig.update_traces(mode='markers+lines')
dr_fig.update_layout(
    title="Crude death rate of Ca Mau",
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Crude death rate",
    showlegend=False,
)

### Mixing matrix

In [ ]:
print(f"Target region: {region.upper()} ({iso3})")
print(f"Proxy country: {source_iso3}")
print("Always age-adjusted to target population")

In [ ]:
update_params = {
    'cdr_adjustment': 0.8003452159878636,
    'contact_rate': 0.013414102898074345,
    'gender.adjustments.infection.male': 2.6408657914674176,
    'infect_death_rate_dict.smear_negative': 0.027588310343242016,
    'infect_death_rate_dict.smear_positive': 0.364337776897486,
    'progression_multiplier': 1.1,
    'rr_infection_latent': 0.20278196465900813,
    'rr_infection_recovered': 0.21190687223342505,
    'self_recovery_rate_dict.smear_positive': 0.20344728302826143,
    'self_recovery_rate_dict.smear_negative': 0.22723824998716693,
    'start_population_size': 267252.06827576435
}
params = baseline_params.update(update_params,calibration_format=True)
model_0  =  p.build_model(params.to_dict())
model_0.run(params.to_dict())
derived_df_0 = model_0.get_derived_outputs_df()

In [ ]:
model_0._stratifications

In [ ]:
model_0._mixing_matrices

In [ ]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [ ]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Percentage latent"
)
latent_plot.show()

In [ ]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notifications",
)
notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
notif_2.update_traces(marker=dict(color="red"))
notif_plot = go.Figure(
    data=notif_1.data + notif_2.data,
)
notif_plot.update_layout(
    title="Modelled vs Data (with James' equation)", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)
notif_plot.show()

In [ ]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="prevalence_infectious",
)
notif_1.show()
# notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
# notif_2.update_traces(marker=dict(color="red"))
# notif_plot = go.Figure(
#     data=notif_1.data + notif_2.data,
# )
# notif_plot.update_layout(
#     title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
# )
# notif_plot.show()

In [1]:
from autumn.core.inputs import get_death_rates_by_agegroup

c:\tools\Anaconda3\envs\autumn\lib\site-packages\summer\runner\vectorized_runner.py:363: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_strain_infection_values(


In [2]:
death_rates_by_age, death_rate_years = get_death_rates_by_agegroup([0, 5, 15, 35, 50], "VNM")

In [7]:
death_rates_by_age

{0: [0.04891194320512563,
  0.033918668027576884,
  0.024432727316924446,
  0.019408181689238014,
  0.018148370056426787,
  0.014535299525724738,
  0.013035498777442808,
  0.011254176500418787,
  0.00931061542295055,
  0.006118489529305104,
  0.005678373734277731,
  0.005333479069860507,
  0.004783694403203202,
  0.0043347634430972014],
 5: [0.005654833192668025,
  0.005297274566250849,
  0.0044664893264966974,
  0.003846307644222109,
  0.005107979016750706,
  0.002711430406000275,
  0.002147435231479338,
  0.0016691038222857585,
  0.0013164357375949395,
  0.0010181764308804923,
  0.0007447183497013854,
  0.0005658632755602,
  0.0005441317519979431,
  0.0005100562239187423],
 15: [0.0030517485729701097,
  0.002700755057428503,
  0.002489838533286184,
  0.002584612327589561,
  0.004088806859043201,
  0.0023927769572424534,
  0.002209771395787853,
  0.001885328047816437,
  0.0016581044683420282,
  0.0015277277213098862,
  0.001427063312059633,
  0.0012953587483924324,
  0.001235650689199

In [3]:
death_rate_years

[1952.5,
 1957.5,
 1962.5,
 1967.5,
 1972.5,
 1977.5,
 1982.5,
 1987.5,
 1992.5,
 1997.5,
 2002.5,
 2007.5,
 2012.5,
 2017.5]